# Install Requirements

In [1]:
!git clone https://github.com/Building-ML-Pipelines/building-machine-learning-pipelines

Cloning into 'building-machine-learning-pipelines'...
remote: Enumerating objects: 77, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 740 (delta 31), reused 36 (delta 13), pack-reused 663
Receiving objects: 100% (740/740), 29.41 MiB | 19.60 MiB/s, done.
Resolving deltas: 100% (388/388), done.


In [2]:
%cd drive/My\ Drive/Building\ ML\ Pipelines/
!pip install -r requirements.txt
%cd ..
%cd ..

/content/drive/My Drive/Building ML Pipelines
     |████████████████████████████████| 1.8MB 5.5MB/s 
     |████████████████████████████████| 153kB 39.8MB/s 
     |████████████████████████████████| 2.3MB 31.6MB/s 
     |████████████████████████████████| 8.6MB 44.6MB/s 
     |████████████████████████████████| 81kB 7.4MB/s 
     |████████████████████████████████| 153kB 44.0MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 1.6MB 41.4MB/s 
     |████████████████████████████████| 63.8MB 61kB/s 
     |████████████████████████████████| 1.8MB 39.7MB/s 
     |████████████████████████████████| 1.5MB 43.7MB/s 
     |████████████████████████████████| 2.8MB 27.5MB/s 
     |████████████████████████████████| 378kB 45.9MB/s 
     |████████████████████████████████| 1.3MB 41.8MB/s 
     |████████████████████████████████| 92kB 10.1MB/s 
     |████████████████████████████████| 829kB 40.1MB/s 
     |████████████████████████████████| 153kB 46.0MB/s 
     |███

/content/drive/My Drive
/content/drive


# Chapter 2: Introduction to TensorFlow Extended

# Basic Interactive Pipeline

In [ ]:
import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

In [ ]:
# Create context object and set up simple in-memory ML MetadataStore via InteractiveContext
context = InteractiveContext()

In [ ]:
from tfx.components import StatisticsGen

In [ ]:
# Set up pipeline component (e.g. StatisticsGen)
statistics_gen = StatisticsGen(examples=examples_gen.outputs["examples"])

# execute each component through run function of context object
context.run(statistics_gen)

In [ ]:
# Visualize StatisticsGen component to inspect features of the dataset
context.show(statistics_gen.outputs["statistics"])

In [ ]:
# Inspect output artifacts of component programmatically
for artifact in statistics_gen.outputs["statistics"].get():
    print(artifact.uri)

# Introduction to Apache Beam

## Setup

In [ ]:
### Get Google Storage file via gsutil
!gsutil cp gs://dataflow-samples/shakespeare/kinglear.txt "drive/My Drive/Building ML Pipelines/Data/" 

Copying gs://dataflow-samples/shakespeare/kinglear.txt...
/ [1 files][153.6 KiB/153.6 KiB]                                                
Operation completed over 1 objects/153.6 KiB.                                    


In [ ]:
import re
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, SetupOptions

In [ ]:
base_dir = "drive/My Drive/Building ML Pipelines/"
input_file = base_dir + "Data/kinglear.txt"
output_file = base_dir + "Chapter 2/Outputs/first test.txt"

pipeline_options = PipelineOptions()

# Use the context manager to define the pipeline
with beam.Pipeline(options=pipeline_options) as p:
    # Read the text into a PCollection
    lines = p | beam.io.ReadFromText(input_file)


    # Apply transformation operations on above read file lines
    counts = (
        lines
        | "Split" >> beam.FlatMap(lambda x: re.findall(r'[A-Za-z\']+', x))
        | "PairWithOne" >> beam.Map(lambda x: (x, 1))
        | "GroupAndSum" >> beam.CombinePerKey(sum)
    )

    def format_result(word_count):
        """
        Convert tuples (token, count) into a String
        """
        (word, count) = word_count
        return "{}: {}".format(word, count)

    # executes format_result function with each element of counts as input
    outputs = counts | "Format" >> beam.Map(format_result)

    # Write the output to the file output_file
    outputs | beam.io.WriteToText(output_file)